In [94]:
import pandas as pd, numpy as np
import math

In [95]:
df = pd.read_csv('poke_genie_export.csv')

In [96]:
# You give a whole evolutionary line here.
mons = ['Spheal', 'Sealeo','Walrein']
mon = mons[-1]
base_ivs = {
    # base_attack, base_defense, base_stamina
    'Walrein': (182, 176, 242),
    'Trevenant': (201,154,198)
}[mon]
df = df[df.Name.isin(mons)]

In [97]:
df

,Index,Name,Form,Pokemon Number,Gender,CP,HP,Atk IV,Def IV,Sta IV,...,Sha/Pur (U),Rank % (L),Rank # (L),Stat Prod (L),Dust Cost (L),Candy Cost (L),Name (L),Form (L),Sha/Pur (L),Marked for PvP use
0,1,Spheal,NaN,363,♀,529,113,10.0,1.0,9.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Spheal,NaN,363,NaN,212,73,4.0,8.0,13.0,...,0.0,92.55%,306.0,96.41%,42000.0,44.0,Spheal,NaN,0.0,NaN
2,3,Spheal,NaN,363,NaN,233,78,2.0,8.0,14.0,...,0.0,75.58%,1001.0,94.46%,40000.0,42.0,Spheal,NaN,0.0,NaN
3,4,Spheal,NaN,363,♂,397,100,1.0,4.0,5.0,...,0.0,83.03%,696.0,95.23%,23400.0,21.0,Spheal,NaN,0.0,NaN
4,5,Spheal,NaN,363,♂,229,77,4.0,2.0,11.0,...,0.0,73.26%,1096.0,94.46%,3300.0,29.0,Sealeo,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,352,Spheal,NaN,363,NaN,516,109,3.0,13.0,3.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352,353,Spheal,NaN,363,NaN,665,129,5.0,2.0,8.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
353,354,Spheal,NaN,363,NaN,165,61,7.0,10.0,4.0,...,0.0,75.78%,993.0,94.51%,0.0,125.0,Walrein,NaN,0.0,NaN
354,355,Spheal,NaN,363,NaN,142,55,10.0,9.0,2.0,...,0.0,84.03%,655.0,95.49%,1200.0,127.0,Walrein,NaN,0.0,NaN


In [98]:
#df.columns

In [99]:
# I definitely stole this table from somewhere, and don't know where.
# If I remember, I'll give proper attribution.
def get_cpm(level):
    d = {
        1:	0.094,
        1.5: 0.1351374318,
        2: 0.16639787,
        2.5: 0.192650919,
        3: 0.21573247,
        3.5: 0.2365726613,
        4: 0.25572005,
        4.5: 0.2735303812,
        5: 0.29024988,
        5.5: 0.3060573775,
        6: 0.3210876,
        6.5: 0.3354450362,
        7: 0.34921268,
        7.5: 0.3624577511,
        8: 0.3752356,
        8.5: 0.387592416,
        9: 0.39956728,
        9.5: 0.4111935514,
        10: 0.4225,
        10.5: 0.4329264091,
        11: 0.44310755,
        11.5: 0.4530599591,
        12: 0.4627984,
        12.5: 0.472336093,
        13: 0.48168495,
        13.5: 0.4908558003,
        14: 0.49985844,
        14.5: 0.508701765,
        15: 0.51739395,
        15.5: 0.5259425113,
        16: 0.5343543,
        16.5: 0.5426357375,
        17: 0.5507927,
        17.5: 0.5588305862,
        18: 0.5667545,
        18.5: 0.5745691333,
        19: 0.5822789,
        19.5: 0.5898879072,
        20: 0.5974,
        20.5: 0.6048236651,
        21: 0.6121573,
        21.5: 0.6194041216,
        22: 0.6265671,
        22.5: 0.6336491432,
        23: 0.64065295,
        23.5: 0.6475809666,
        24: 0.65443563,
        24.5: 0.6612192524,
        25: 0.667934,
        25.5: 0.6745818959,
        26: 0.6811649,
        26.5: 0.6876849038,
        27: 0.69414365,
        27.5: 0.70054287,
        28: 0.7068842,
        28.5: 0.7131691091,
        29: 0.7193991,
        29.5: 0.7255756136,
        30: 0.7317,
        30.5: 0.7347410093,
        31: 0.7377695,
        31.5: 0.7407855938,
        32: 0.74378943,
        32.5: 0.7467812109,
        33: 0.74976104,
        33.5: 0.7527290867,
        34: 0.7556855,
        34.5: 0.7586303683,
        35: 0.76156384,
        35.5: 0.7644860647,
        36: 0.76739717,
        36.5: 0.7702972656,
        37: 0.7731865,
        37.5: 0.7760649616,
        38: 0.77893275,
        38.5: 0.7817900548,
        39: 0.784637,
        39.5: 0.7874736075,
        40: 0.7903,
        40.5: 0.792803968,
        41: 0.79530001,
        }
    return d[level]

In [100]:
get_cpm(2)

0.16639787

In [101]:
def ivs_to_stats(my_a, my_d, my_s,max_level=40,max_cp=1500.99, 
                 mon=mon, base_ivs=base_ivs):
    base_attack, base_defense, base_stamina = base_ivs
    attack = base_attack + my_a
    defense = my_d + base_defense
    stamina = my_s + base_stamina
    level = 10
    cp = 10
    level_attack, level_defense, level_stamina, stat_prod = 0,0,0,0
    stats = (level, cp, level_attack, level_defense, level_stamina)
    while level <= max_level:
        cpm = get_cpm(level)
        cp = (attack * defense**0.5 * stamina**0.5 * cpm**2) / 10
        level_attack = attack * cpm
        level_defense = defense * cpm
        level_stamina = stamina * cpm
        if cp <= max_cp:
            stat_prod = math.floor(level_attack*level_defense*math.floor(level_stamina))
            stats = level, cp, level_attack, level_defense, level_stamina, stat_prod
        level = level + 0.5
    level, cp, level_attack, level_defense, level_stamina, stat_prod  = stats
    #print(f'{level_attack}, {level_defense}, {level_stamina}, {level_attack*level_defense*level_stamina}')
    return stats
    #print(f'level {level} cp {cp:.0f} attack {level_attack:.1f} defense {level_defense:.1f} stamina {level_stamina:.0f}')

In [102]:
ivs_to_stats(4,14,2)
ivs_to_stats(4,9,8)
ivs_to_stats(2,9,2)
ivs_to_stats(4,10,7)

(21.0, 1500.013458082344, 113.8612578, 113.8612578, 152.4271677, 1970586)

In [103]:
d = {'CP':[],'max CP':[],'level':[],'attack':[],'defense':[],'stamina':[],'a':[],'d':[],'s':[],'statprod':[]}
for row in df.iterrows():
    i,s = row
    orig_cp, my_a, my_d, my_s = s.CP, s['Atk IV'], s['Def IV'], s['Sta IV']
    level, cp, attack, defense, stamina, stat_prod = ivs_to_stats(my_a, my_d, my_s,max_level=41)
    #print('orig_cp',orig_cp,'my_a',my_a)
    d['CP'].append(orig_cp)
    d['max CP'].append(int(np.floor(cp)))
    d['level'].append(level)
    d['attack'].append(attack)
    d['defense'].append(defense)
    d['stamina'].append(int(np.floor(stamina)))
    d['a'].append(my_a)
    d['d'].append(my_d)
    d['s'].append(my_s)
    d['statprod'].append(stat_prod)
mine = pd.DataFrame.from_dict(d)

In [104]:
# main GL
attack,defense,hp = 113.77, 113, 148
print('Here are the ones that meet the attack breakpoint')
these = mine[mine.attack >= attack]
these.sort_values('statprod',ascending=False)

Here are the ones that meet the attack breakpoint


,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
177,177,1496,21.5,113.970358,110.873338,155,2.0,3.0,9.0,1958622
204,147,1498,22.0,114.035212,111.528944,154,0.0,2.0,5.0,1958606
61,209,1498,21.0,114.473415,108.963999,157,5.0,2.0,15.0,1958336
154,203,1500,20.0,114.700800,112.311200,152,10.0,12.0,13.0,1958092
176,11,1497,21.0,114.473415,113.249100,151,5.0,9.0,5.0,1957565
...,...,...,...,...,...,...,...,...,...,...
175,112,1468,20.5,116.730967,107.658612,146,11.0,2.0,1.0,1834795
29,553,1475,20.5,117.940615,106.448965,146,13.0,0.0,1.0,1832979
308,143,1465,20.0,117.090400,105.739800,148,14.0,1.0,6.0,1832405
119,169,1466,20.0,117.090400,106.337200,147,14.0,2.0,5.0,1830306


In [105]:
# GL GOD TIER
attack, defense, hp = 114.46,114.75, 148
print('Here are the ones that hit all of the BPs')
these = mine[mine.attack >= attack]
these = these[these.defense >= defense]
these = these[these.stamina >= hp]
these.sort_values('statprod',ascending=False)

Here are the ones that hit all of the BPs


,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


In [106]:
attack,defense,hp = 113.77, 113, 148
print('Here are the ones that hit all of the BPs')
these = mine[mine.attack >= attack]
these = these[these.defense >= defense]
these = these[these.stamina >= hp]
these.sort_values('statprod',ascending=False)

Here are the ones that hit all of the BPs


,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
176,11,1497,21.0,114.473415,113.249100,151,5.0,9.0,5.0,1957565
21,229,1497,20.0,114.103400,113.506000,151,9.0,14.0,12.0,1955664
79,590,1495,21.0,113.861258,115.085572,149,4.0,12.0,3.0,1952464
190,395,1499,20.5,114.916496,113.102025,150,8.0,11.0,7.0,1949593
353,316,1493,21.5,113.970358,114.589762,149,2.0,9.0,0.0,1945915
23,295,1494,20.5,114.311673,114.916496,148,7.0,14.0,4.0,1944171
149,111,1495,21.0,114.473415,113.861258,149,5.0,10.0,3.0,1942078
98,385,1494,20.0,114.700800,114.103400,148,10.0,15.0,7.0,1936987
164,116,1493,20.5,114.916496,113.102025,149,8.0,11.0,5.0,1936595
84,178,1493,21.5,114.589762,113.350954,149,3.0,7.0,0.0,1935339


In [107]:
# GL high attack
attack, defense, hp = 114.46,113, 148
print('Here are the ones that hit all of the BPs')
these = mine[mine.attack >= attack]
these = these[these.defense >= defense]
these = these[these.stamina >= hp]
these.sort_values('statprod',ascending=False)

Here are the ones that hit all of the BPs


,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
176,11,1497,21.0,114.473415,113.249100,151,5.0,9.0,5.0,1957565
190,395,1499,20.5,114.916496,113.102025,150,8.0,11.0,7.0,1949593
149,111,1495,21.0,114.473415,113.861258,149,5.0,10.0,3.0,1942078
98,385,1494,20.0,114.700800,114.103400,148,10.0,15.0,7.0,1936987
164,116,1493,20.5,114.916496,113.102025,149,8.0,11.0,5.0,1936595
84,178,1493,21.5,114.589762,113.350954,149,3.0,7.0,0.0,1935339
163,406,1493,21.5,114.589762,113.350954,149,3.0,7.0,0.0,1935339
338,468,1488,21.0,114.473415,113.249100,149,5.0,9.0,2.0,1931637
231,35,1489,21.0,114.473415,113.861258,148,5.0,10.0,1.0,1929044


In [108]:
# GL waterfall
attack, defense, hp = 114.62, 113, 148
print('Here are the ones that hit all of the BPs')
these = mine[mine.attack >= attack]
these = these[these.defense >= defense]
these = these[these.stamina >= hp]
these.sort_values('statprod',ascending=False)

Here are the ones that hit all of the BPs


,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
190,395,1499,20.5,114.916496,113.102025,150,8.0,11.0,7.0,1949593
98,385,1494,20.0,114.700800,114.103400,148,10.0,15.0,7.0,1936987
164,116,1493,20.5,114.916496,113.102025,149,8.0,11.0,5.0,1936595


# UL requires recalculating

In [116]:
d = {'CP':[],'max CP':[],'level':[],'attack':[],'defense':[],'stamina':[],'a':[],'d':[],'s':[],'statprod':[]}
for row in df.iterrows():
    i,s = row
    orig_cp, my_a, my_d, my_s = s.CP, s['Atk IV'], s['Def IV'], s['Sta IV']
    level, cp, attack, defense, stamina, stat_prod = ivs_to_stats(my_a, my_d, my_s,max_level=41,max_cp=2500.99)
    #print('orig_cp',orig_cp,'my_a',my_a)
    d['CP'].append(orig_cp)
    d['max CP'].append(int(np.floor(cp)))
    d['level'].append(level)
    d['attack'].append(attack)
    d['defense'].append(defense)
    d['stamina'].append(int(np.floor(stamina)))
    d['a'].append(my_a)
    d['d'].append(my_d)
    d['s'].append(my_s)
    d['statprod'].append(stat_prod)
mine = pd.DataFrame.from_dict(d)

In [117]:
# UL supreme
attack, defense, hp = 147, 145.3, 197
print('Here are the ones that hit all of the BPs')
these = mine[mine.attack >= attack]
these = these[these.defense >= defense]
these = these[these.stamina >= hp]
these.sort_values('statprod',ascending=False)

Here are the ones that hit all of the BPs


,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


In [118]:
# UL the other ten
attack, defense, hp = 147.76, 143.89, 197
print('Here are the ones that hit all of the BPs')
these = mine[mine.attack >= attack]
these = these[these.defense >= defense]
these = these[these.stamina >= hp]
these.sort_values('statprod',ascending=False)

Here are the ones that hit all of the BPs


,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
123,494,2494,38.0,147.997222,144.102559,197,8.0,9.0,11.0,4201375


In [125]:
# UL hosing around
attack, defense, hp = 147.75, 145, 0
print('Here are the ones that hit all of the BPs')
these = mine[mine.attack >= attack]
these = these[these.defense >= defense]
these = these[these.stamina >= hp]
these.sort_values('statprod',ascending=False)

Here are the ones that hit all of the BPs


,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
56,61,2500,40.5,148.254342,145.875930,195,5.0,8.0,4.0,4217214
23,295,2497,38.5,147.758320,148.540110,192,7.0,14.0,4.0,4214023
176,11,2496,40.0,147.786100,146.205500,195,5.0,9.0,5.0,4213392
188,723,2497,38.0,147.997222,147.997222,192,8.0,14.0,5.0,4205410
249,398,2500,35.5,148.310297,145.252352,195,12.0,14.0,14.0,4200771
85,292,2500,35.5,148.310297,145.252352,195,12.0,14.0,14.0,4200771
22,300,2496,41.0,147.925802,145.539902,195,4.0,7.0,4.0,4198175
231,35,2498,40.5,148.254342,147.461538,192,5.0,10.0,1.0,4197468
338,468,2497,40.5,148.254342,146.668734,193,5.0,9.0,2.0,4196645
149,111,2493,40.0,147.786100,146.995800,193,5.0,10.0,3.0,4192719


In [111]:
# ML
#Want 15/14/13
print('Here are the ones that hit all of the BPs')
a,d,s = 15,14,13
these = mine[mine.a >= a]
these = these[these.d >= d]
these = these[these.s >= s]
these.sort_values('statprod',ascending=False)

Here are the ones that hit all of the BPs


,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
234,1557,1479,19.0,114.708943,111.215270,149,15.0,15.0,15.0,1900850
243,408,1470,19.0,114.708943,110.632991,148,15.0,14.0,13.0,1878207


In [ ]:
# gfisk attack bp
these = pht[pht.attack >= 167.98]
# hp breakpoint for mirror
these = these[these.stamina >= 167]
these

In [ ]:
# mew attack bp
these = pht[pht.attack >= 127.8]
these = these[these.defense >= 104.9] # best. next is 104.9.
# hp breakpoint for mirror
these = these[these.stamina >= 128]
these # 3/15/7 is RS, I think maybe 3/14/11 for Azu

In [ ]:
# mew attack bp
these = pht[pht.attack >= 127] 
these = these[these.defense >= 104.9] # best. next is 104.9.
# hp breakpoint for mirror
these = these[these.stamina >= 128]
these

In [ ]:
ivs_to_stats(my_a=3,my_d=14,my_s=11,max_cp=1501)